In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
os.chdir("../..")
# from parc.data import EnergeticMatDataPipeLine as EmData
from parc import misc, metrics, model,visualization
from parc.model import model_burgers
from skimage.measure import block_reduce


2024-01-04 15:32:45.246837: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data pipeline

In [2]:
import time
import os
import numpy as np
import skimage
from skimage.measure import block_reduce
# 
R_list = [1000, 2500, 5000, 7500, 10000]
a_list = [0.5, 0.6, 0.7, 0.8, 0.9]
w_list = [0.7, 0.8, 0.9, 1.0]

def clip_raw_data(idx_range, sequence_length=2, n_state_var=3, purpose = "diff_training"):
    vel_seq_whole = []

    for R in R_list:
        for a in a_list:
            for w in w_list:
                data_file_name = 'burgers_train_' + str(int(R)) + '_' + str(int(a*10)) + '_' + str(int(w*10)) + '.npy'
#                 print(data_file_name)
                file_path = './train_data/' + data_file_name                
                if os.path.exists(file_path):
                    raw_data = np.float32(np.load(file_path))
                    raw_data = np.moveaxis(raw_data,-2,0)
#                     print(raw_data.shape)
#                     raw_data = np.clip(raw_data,0,1.0)
                    data_shape = raw_data.shape
                    num_time_steps = data_shape[0]
#                     print(num_time_steps)
                    norm_r = R/15000
                    r_img = norm_r*np.ones(shape = (1,data_shape[1],data_shape[2],1))
#                     print(r_img.shape)
                                      
                    vel_seq_case = [np.concatenate(
                        [np.concatenate([np.expand_dims(raw_data[(j + k), :, :, :],axis = 0),r_img],axis = -1)
                        for k in range(sequence_length)], axis=-1)
                        for j in range (num_time_steps-sequence_length+1)] 
#                     print(vel_seq_case.shape)
                    vel_seq_whole.extend(vel_seq_case)

    vel_seq_whole = np.concatenate(vel_seq_whole, axis=0)

    return vel_seq_whole

seq_clipped = clip_raw_data((1,100),2)



# Model definition

In [3]:
from tensorflow import keras
from tensorflow.keras import  layers, regularizers
from keras.layers import *
import tensorflow as tf
from parc import layer

from tensorflow.keras.layers import Concatenate, Input
from tensorflow.keras.models import Model

"""
Model definition customized for Burgers' problems: 
    - In this problem, state vars and velocity only include velocity
    - Architecture was adjusted to make it lighter and comparable with PhyCRNet
"""

def neural_ode_differentiator_burgers():
    # U-Net backbone
    feature_extraction = layer.feature_extraction_burgers(input_shape = (64,64), n_channel = 3)
    velocity_field = Input(shape=(64,64, 3), dtype = tf.float32)
    dynamic_feature = feature_extraction(velocity_field)
    # Final mapping
    velocity_dot = Conv2D(2, 1, padding="same")(dynamic_feature)
    
    differentiator = Model(velocity_field, velocity_dot)
    return differentiator

def integrator_burgers():
    velocity_integrator = layer.integrator_cnn(input_shape = (128,128), n_base_features = 64, n_output=2)

    velocity_prev = keras.layers.Input(shape = (128,128, 2), dtype = tf.float32)
    velocity_dot = keras.layers.Input(shape = (128,128, 2), dtype = tf.float32)

    velocity_next = velocity_integrator([velocity_dot, velocity_prev])
    integrator = keras.Model([velocity_dot, velocity_prev], [velocity_next])
    return integrator

class PARCv2_burgers(keras.Model):
    def __init__(self, n_time_step, step_size, solver = "rk4", mode = "integrator_training", use_data_driven_int = True, differentiator_backbone = 'em', **kwargs):
        super(PARCv2_burgers, self).__init__(**kwargs)
        self.n_time_step = n_time_step
        self.step_size = step_size
        self.solver = solver
        self.mode = mode
        self.use_data_driven_int = use_data_driven_int
        self.differentiator = neural_ode_differentiator_burgers()
        self.integrator = integrator_burgers()
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        if self.mode == "integrator_training":
            self.differentiator.trainable = False
        else:
            self.integrator.trainable = False

    @property
    def metrics(self):
        return [
        self.total_loss_tracker,
        ]
    
    def call(self, input_tensor):
        input_seq_current = tf.cast(input_tensor,dtype = tf.float32)
        res = [] 
        res.append(input_seq_current)
        for _ in range(self.n_time_step):    
            velocity_next, update = self.explicit_update(input_seq_current)
            input_seq_current = velocity_next
            res.append(input_seq_current)
        output = tf.concat(res,axis = -1)
        return output

    @tf.function
    def train_step(self, data):
        velocity_init = tf.cast(data[0], dtype = tf.float32)
        velocity_gt = tf.cast(data[1], dtype = tf.float32)

        input_seq_current = velocity_init
        with tf.GradientTape() as tape:
            if self.mode == "integrator_training":
                for ts in range(self.n_time_step):
                    # Compute k1
                    input_seq_current, update = self.explicit_update(input_seq_current)
                    velocity_next = self.integrator([update, input_seq_current])
                    input_seq_current = Concatenate(axis = -1)([velocity_next,velocity_init[:,:,:,2:]])
                
            else: 
                velocity_next, update = self.explicit_update(input_seq_current)

            total_loss  = tf.keras.losses.MeanAbsoluteError(reduction = 'sum')(velocity_next[:,:,:,:2],velocity_gt[:,:,:,:2])
                           
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        self.total_loss_tracker.update_state(total_loss)

        return {
            "total_loss": self.total_loss_tracker.result(),
        }
    
    # Update scheme
    def explicit_update(self, input_seq_current):
        input_seq_current = tf.clip_by_value(input_seq_current, 0, 1)

        if self.solver == "rk4":
            input_seq_current, update = self.rk4_update(input_seq_current)
        elif self.solver == 'heun':
            input_seq_current, update = self.heun_update(input_seq_current)
        else:
            input_seq_current, update = self.euler_update(input_seq_current)

        return input_seq_current, update

    def rk4_update(self, input_seq_current):

        # Compute k1
        k1 = self.differentiator(input_seq_current)

        # Compute k2
        inp_k2 = input_seq_current[:,:,:,:2] + self.step_size*1/2*k1 
        inp_k2 = Concatenate(axis = -1)([inp_k2,input_seq_current[:,:,:,2:]])

        k2 = self.differentiator(inp_k2)

        # Compute k3
        inp_k3 = input_seq_current[:,:,:,:2] + self.step_size*1/2*k2
        inp_k3 = Concatenate(axis = -1)([inp_k3,input_seq_current[:,:,:,2:]])
        k3 = self.differentiator(inp_k3)

        # Compute k4
        inp_k4 = input_seq_current[:,:,:,:2] + self.step_size*k3
        inp_k4 = Concatenate(axis = -1)([inp_k4,input_seq_current[:,:,:,2:]])

        k4 = self.differentiator(inp_k4)

        # Final
        update = 1/6*(k1 + 2*k2 + 2*k3 + k4)
        final_state = input_seq_current[:,:,:,:2] + self.step_size*update 
        input_seq_current = Concatenate(axis = -1)([final_state,input_seq_current[:,:,:,2:]])
        return input_seq_current, update
    
    # Euler update function
    def heun_update(self, input_seq_current):
        # Compute update
        k1 = self.differentiator(input_seq_current)

        # Compute k2
        inp_k2 = input_seq_current[:,:,:,:2] + self.step_size*k1 
        inp_k2 = Concatenate(axis = -1)([inp_k2,input_seq_current[:,:,:,2:]])

        k2 = self.differentiator(inp_k2)
        
        update = 1/2*(k1 + k2)

        final_states = input_seq_current[:,:,:,:2] + self.step_size*update 
        input_seq_current = Concatenate(axis = -1)([final_states,input_seq_current[:,:,:,2:]])

        return input_seq_current, update
    
    # Euler update function
    def euler_update(self, input_seq_current):
        # Compute update
        update = self.differentiator(input_seq_current)
        input_seq_current = input_seq_current + self.step_size*update 

        return input_seq_current, update

# Training


### Stage 1: Differentiator training

In [4]:
# Create tf.dataset
dataset_input = tf.data.Dataset.from_tensor_slices(seq_clipped[:,:,:,:3])
dataset_label = tf.data.Dataset.from_tensor_slices(seq_clipped[:,:,:,3:])
dataset = tf.data.Dataset.zip((dataset_input, dataset_label))
dataset = dataset.shuffle(buffer_size = 10000) 
dataset = dataset.batch(16)

2024-01-04 15:32:52.623686: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78791 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:4e:00.0, compute capability: 8.0


In [ ]:
tf.keras.backend.clear_session()
parc = PARCv2_burgers(n_time_step = 1, step_size= 1/100, solver = "heun", mode = "differentiator_training")
parc.differentiator.load_weights('parc_diff_burgers_heun.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.000075, beta_1 = 0.9, beta_2 = 0.99))
parc.fit(dataset, epochs = 50, shuffle = True)

Epoch 1/50


2024-01-04 15:32:56.873901: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2024-01-04 15:32:59.111719: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f910e2f64e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-04 15:32:59.111769: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2024-01-04 15:32:59.208612: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-04 15:32:59.991430: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


625/625 [==============================] - 15s 12ms/step - total_loss: 0.9138
Epoch 2/50
625/625 [==============================] - 7s 11ms/step - total_loss: 0.7546
Epoch 3/50
625/625 [==============================] - 7s 11ms/step - total_loss: 0.7731
Epoch 4/50
625/625 [==============================] - 7s 11ms/step - total_loss: 0.7136
Epoch 5/50
625/625 [==============================] - 7s 11ms/step - total_loss: 0.7808
Epoch 6/50
625/625 [==============================] - 7s 11ms/step - total_loss: 0.7459
Epoch 7/50
625/625 [==============================] - 7s 11ms/step - total_loss: 0.7291
Epoch 8/50
625/625 [==============================] - 7s 11ms/step - total_loss: 0.7708
Epoch 9/50
625/625 [==============================] - 7s 11ms/step - total_loss: 0.7291
Epoch 10/50
625/625 [==============================] - 7s 11ms/step - total_loss: 0.7783
Epoch 11/50
625/625 [==============================] - 7s 11ms/step - total_loss: 0.7199
Epoch 12/50
625/625 [===================

In [ ]:
parc.differentiator.save_weights('parc_diff_burgers_heun_2.h5')

# Validation

In [ ]:
R_list = [100, 500, 3000, 6500, 12500, 15000]
a_list = [0.35, 0.40, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.00]
w_list = [0.55, 0.6, 0.65, 0.75, 0.85, 0.95, 1.05]

def clip_raw_data_for_validation(idx_range, sequence_length=2, n_state_var=3, purpose = "diff_training"):
    vel_seq_whole = []

    for R in R_list:
        for a in a_list:
            for w in w_list:
                data_file_name = 'burgers_test_' + str(int(R)) + '_' + str(int(a*100)) + '_' + str(int(w*100)) + '.npy'
                print(data_file_name)

                file_path = './test_data/' + data_file_name
                if os.path.exists(file_path):
#                     print(data_file_name)

                    raw_data = np.float32(np.load(file_path))
                    raw_data = np.moveaxis(raw_data,-2,0)

#                     raw_data = skimage.measure.block_reduce(raw_data[:,:,:,:], (1,,2,1),np.max)
#                     raw_data = np.moveaxis(raw_data, 1, -1)

                    data_shape = raw_data.shape
                    num_time_steps = data_shape[0]
#                     print(num_time_steps)
                    norm_r = R/15000
                    r_img = norm_r*np.ones(shape = (1,data_shape[1],data_shape[2],1))
#                     print(r_img.shape)
#                     print(r_img.shape)
                                      
                    vel_seq_case = [np.concatenate(
                        [np.concatenate([np.expand_dims(raw_data[(j + k), :, :, :],axis = 0),r_img],axis = -1)
                        for k in range(sequence_length)], axis=-1)
                        for j in range (1)] 
                    vel_seq_whole.extend(vel_seq_case)
    print(len(vel_seq_whole))
    vel_seq_whole = np.concatenate(vel_seq_whole, axis=0)

    return vel_seq_whole

seq_clipped_test = clip_raw_data_for_validation((1,100),100)
# vel_seq_norm_test = data_normalization_test(seq_clipped_test, vel_seq_norm[1], vel_seq_norm[2], 2)

In [ ]:
tf.keras.backend.clear_session()
parc = PARCv2_burgers(n_time_step = 99, step_size= 1/100, solver = "heun")
parc.differentiator.load_weights('parc_diff_burgers_heun_2.h5')

In [9]:
prediction_data = []
for j in range(378):
    input_seq_current = tf.cast(seq_clipped_test[j:j+1,:,:,:3], dtype = tf.float32)
    res = parc.predict(input_seq_current)
    prediction_data.append(res)
    print('Finish case ', j)
prediction_data = np.concatenate(prediction_data, axis = 0)

2024-01-04 15:21:00.907709: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:625] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 [==============================] - 18s 18s/step
Finish case  0
1/1 [==============================] - 0s 146ms/step
Finish case  1
1/1 [==============================] - 0s 150ms/step
Finish case  2
1/1 [==============================] - 0s 145ms/step
Finish case  3
1/1 [==============================] - 0s 149ms/step
Finish case  4
1/1 [==============================] - 0s 145ms/step
Finish case  5
1/1 [==============================] - 0s 142ms/step
Finish case  6
1/1 [==============================] - 0s 147ms/step
Finish case  7
1/1 [==============================] - 0s 144ms/step
Finish case  8
1/1 [==============================] - 0s 145ms/step
Finish case  9
1/1 [==============================] - 0s 147ms/step
Finish case  10
1/1 [==============================] - 0s 148ms/step
Finish case  11
1/1 [==============================] - 0s 149ms/step
Finish case  12
1/1 [==============================] - 0s 148ms/step
Finish case  13
1/1 [==============================] - 0s 149

In [10]:
prediction_data.shape

(378, 64, 64, 300)

In [10]:
np.save('./plotting/burgers/neuralode_burgers.npy',prediction_data)